In [ ]:
# composite image generation by copy-and-paste followed by image blending.
from libcom import get_composite_image
from libcom.utils.process_image import make_image_grid
import cv2
bg_img  = '../tests/source/background/8351494f79f6bf8f_m03k3r_73176af9_08.png'
bbox = [223, 104, 431, 481] # x1,y1,x2,y2
fg_img  = '../tests/source/foreground/8351494f79f6bf8f_m03k3r_73176af9_08.png'
fg_mask = '../tests/source/foreground_mask/8351494f79f6bf8f_m03k3r_73176af9_08.png'
# generate composite images by naive methods
img_list = [bg_img, fg_img]
comp_img1, comp_mask1 = get_composite_image(fg_img, fg_mask, bg_img, bbox, 'none')
comp_img2, comp_mask2 = get_composite_image(fg_img, fg_mask, bg_img, bbox, 'gaussian')
comp_img3, comp_mask3 = get_composite_image(fg_img, fg_mask, bg_img, bbox, 'poisson')
img_list += [comp_img1, comp_mask1, comp_img2, comp_mask2, comp_img3, comp_mask3]
# visualization results
grid_img  = make_image_grid(img_list, cols=4)
cv2.imwrite('../docs/_static/image/generatecomposite_result1.jpg', grid_img)

In [ ]:
# transfer foreground color to fit background scene.
from libcom import color_transfer
from libcom.utils.process_image import make_image_grid
import cv2
comp_img  = '../tests/source/composite/5b9e0751ca458cad_m0gjkl_40736b77_04.png'
comp_mask = '../tests/source/composite_mask/5b9e0751ca458cad_m0gjkl_40736b77_04.png'
trans_img = color_transfer(comp_img, comp_mask)
# visualization results
grid_img  = make_image_grid([comp_img, comp_mask, trans_img])
cv2.imwrite('../docs/_static/image/colortransfer_result1.jpg', grid_img)

In [ ]:
# predict harmony score for a composite image.
from libcom import HarmonyScoreModel
from libcom.utils.process_image import make_image_grid
import cv2
net = HarmonyScoreModel(device=0, model_type='BargainNet')
comp_img  = '../tests/source/composite/1bb218825e370c51_m02p5f1q_4a9db762_30.png'
comp_mask = '../tests/source/composite_mask/1bb218825e370c51_m02p5f1q_4a9db762_30.png'
score     = net(comp_img, comp_mask)
grid_img  = make_image_grid([comp_img, comp_mask], text_list=[f'harmony_score:{score:.2f}', 'composite-mask'])
cv2.imwrite('../docs/_static/image/harmonyscore_result1.jpg', grid_img)

In [ ]:
# Object Placement Assessment (OPA) score prediction
from libcom import OPAScoreModel
from libcom.utils.process_image import make_image_grid
import cv2
net = OPAScoreModel(device=0, model_type='SimOPA')
comp_img  = '../tests/source/composite/1bb218825e370c51_m02p5f1q_4a9db762_30.png'
comp_mask = '../tests/source/composite_mask/1bb218825e370c51_m02p5f1q_4a9db762_30.png'
score = net(comp_img, comp_mask)
grid_img  = make_image_grid([comp_img, comp_mask], text_list=[f'opa_score:{score:.2f}', 'composite-mask'])
cv2.imwrite('../docs/_static/image/opascore_result1.jpg', grid_img)

In [ ]:
# fos_score demo
from libcom.utils.process_image import make_image_grid
from libcom import FOSScoreModel
import cv2
MODEL_TYPE = 'FOS_D' # choose from 'FOS_D', 'FOS_E'
background = '../tests/source/background/f80eda2459853824_m09g1w_b2413ec8_11.png'
fg_bbox    = [175, 82, 309, 310] # x1,y1,x2,y2
foreground = '../tests/source/foreground/f80eda2459853824_m09g1w_b2413ec8_11.png'
foreground_mask = '../tests/source/foreground_mask/f80eda2459853824_m09g1w_b2413ec8_11.png'
net        = FOSScoreModel(device=0, model_type=MODEL_TYPE)
score      = net(background_image, foreground_image, bounding_box, foreground_mask=foreground_mask)
grid_img   = make_image_grid([background_image, foreground_image, composite_image], text_list=[f'fos_score:{score:.2f}'])
cv2.imshow('fos_score_demo', grid_img)

In [ ]:
# controllable image composition
from libcom import ControlComModel
from libcom.utils.process_image import make_image_grid
import cv2
bg_img   = '../tests/source/background/f80eda2459853824_m09g1w_b2413ec8_11.png'
fg_bbox  = [175, 82, 309, 310] # x1,y1,x2,y2
fg_img   = '../tests/source/foreground/f80eda2459853824_m09g1w_b2413ec8_11.png'
fg_mask  = '../tests/source/foreground_mask/f80eda2459853824_m09g1w_b2413ec8_11.png'
net      = ControlComModel(device=0, model_type='ControlCom')
comp_img = net(bg_img, fg_img, fg_bbox, fg_mask, task=['blending', 'harmonization'])
grid_img = make_image_grid([bg_img, fg_img, comp_img[0], comp_img[1]])
cv2.imwrite('../docs/_static/image/controlcom_result1.jpg', grid_img)